In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import keras.utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import keras.regularizers as kr
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import keras.losses as ls
import matplotlib.pyplot as plt
import sys
import pydot
np.set_printoptions(threshold=sys.maxsize)
%matplotlib qt

In [ ]:
###In questa parte si selezionano gli oggetti per il training. In automatico 
###i dati vengono selezionati, eliminati i dati con più di 4 elementi pari a
###zero e vengono selezionate randomicamente delle stelle da usare come 
###training dataset. La dimensione relativa tra train set e validation set
###è controllata dal parametro traindim.

data=pd.read_csv('./Data/DEFMSData.csv')
nancount=[]
ind=[]
for i in range(0,data.shape[0]):
    nancount.append(data.iloc[i,12:].isnull().sum())
    if nancount[i]<5:
        ind.append(i)
    else:
        pass
data=data.iloc[ind,:]
subselect=['NGC2547','NGC2516','NGC6253','M67','NGC6633']
data=data.set_index(['GES_FLD'])
data=data.loc[subselect,:]
l=[0]
l2=[0]
for i in range(0,len(subselect)):
    b=np.sum(l)+data.loc[subselect[i],:].shape[0]
    l2.append(b)
    a=data.loc[subselect[i],:].shape[0]
    l.append(a)
traindim=0.85
subseltrain=[]
subseltest=[]
for j in range(0,len(subselect)):
    subseltrain=np.concatenate((subseltrain,np.random.choice(range(l2[j],l2[j+1]),round(l[j+1]*traindim),replace=False)))
subseltrain=subseltrain.astype(int)
for k in range(0,data.shape[0]):
    if k in subseltrain:
        pass
    else:
        subseltest.append(k)
subseltest=np.array(subseltest)

data=data.reset_index()   
dataset=data.values    

invartrain=dataset[subseltrain,12:].astype(float)
invartrain=np.nan_to_num(invartrain)
outvartrain=dataset[subseltrain,0]
invarfit=dataset[subseltest,12:].astype(float)
invarfit=np.nan_to_num(invarfit)
outvarfit=dataset[subseltest,0]

In [ ]:
###Viene generato il file con le label degli oggetti.

encoder=LabelEncoder()
encoder.fit(outvartrain)
encodedout=encoder.transform(outvartrain)
dummyout=np_utils.to_categorical(encodedout)
encoder.fit(outvarfit)
encout=encoder.transform(outvarfit)
dummytest=np_utils.to_categorical(encout)

In [ ]:
###Modello di rete. La dimensione dei layer varia al variare della dimensione
###dell'input. 

model=Sequential()
model.add(Dense(invartrain.shape[1],input_dim=invartrain.shape[1],activation='selu',kernel_regularizer=kr.l1(0.01)))
model.add(Dense(round(invartrain.shape[1]*2/5),activation='selu',kernel_regularizer=kr.l1(0.01)))
model.add(Dropout(0.5))
model.add(Dense(3,activation='softsign'))
model.add(Dense(dummyout.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
###Training

history=model.fit(invartrain,dummyout,validation_data=(invarfit,dummytest),epochs=10000,batch_size=None)

In [ ]:
###Valutazione del training fatto, i valori finali vengono printati dalla
###cell successiva.

trainmse=model.evaluate(invartrain,dummyout,verbose=0)
testmse=model.evaluate(invarfit,dummytest,verbose=0)

In [ ]:
trainmse,testmse

In [ ]:
###Plot delle Loss

plt.close()
plt.title('Train vs Test Loss Functions')
plt.xlabel('Ages')
plt.ylabel('Loss')
plt.plot(history.history['loss'], label='Train set')
plt.plot(history.history['val_loss'], label='Test set')
#plt.plot(history.history['val_accuracy'],label='accuracy',linestyle='--')
plt.legend()
plt.show()

In [ ]:
###Vengono salvati i weights e bias in 6 file distinti, che verranno caricati nel prossimo file:
###MSFinalNN. NB sono disponibili i weights trovati da me nella cartella FixedWeights.

fixedw=model.get_weights()
l0=model.layers[0].get_weights()
np.save('./EvalWeights/msl0.npy',l0[0])
np.save('./EvalWeights/msl0bias.npy',l0[1])
l1=model.layers[1].get_weights()
np.save('./EvalWeights/msl1.npy',l1[0])
np.save('./EvalWeights/msl1bias.npy',l1[1])
l2=model.layers[3].get_weights()
np.save('./EvalWeights/msl2.npy',l2[0])
np.save('./EvalWeights/msl2bias.npy',l2[1])